# **Data Retriever**

In [23]:
import MetaTrader5 as mt5
import pandas as pd
import ta.momentum as momentum
from datetime import datetime, timedelta
import time
import plotly.graph_objects as go

def get_historical_data(symbol, timeframe, start_date, end_date):
    if not mt5.initialize():
        print("initialize() failed ☢️")
        mt5.shutdown()
        return None

    rates = mt5.copy_rates_range(symbol, timeframe, start_date, end_date)
    print(f"Retrieved {len(rates)} data points")

    if rates is None:
        print("Failed to retrieve historical data. ☢️")
        mt5.shutdown()
        return None

    df = pd.DataFrame(rates)
    df["time"] = pd.to_datetime(df["time"], unit="s")
    df = df.set_index("time")

    mt5.shutdown()

    return df


def calculate_rsi(df, period=14):
    try:
        rsi_indicator = momentum.RSIIndicator(df["close"], window=period)
        df["rsi"] = rsi_indicator.rsi()
    except Exception as e:
        print(f"Error calculating RSI: {e}")

def find_filling_mode(symbol):
    for i in range(2):
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": mt5.symbol_info(symbol).volume_min,
            "type": mt5.ORDER_TYPE_BUY,
            "price": mt5.symbol_info_tick(symbol).ask,
            "type_filling": i,
            "type_time": mt5.ORDER_TIME_GTC
        }

        result = mt5.order_check(request)

        if result.comment == "Done":
            print("Trade was closed")
            break

    return i


# **BackTest + Trade Visualizer + RSI (Maverick boom 1000/500/300)**

In [46]:
from plotly.subplots import make_subplots

def backtest_strategy(df, lot_size=0.2):
    trades = []
    in_trade = False

    for i in range(3, len(df)):
        current_bar = df.iloc[i]
        previous_bar = df.iloc[i - 1]
        confirmation_bar = df.iloc[i - 2]

        # Check for sell conditions
        if current_bar["rsi"] > 70 and confirmation_bar["close"] > confirmation_bar["open"] \
                and previous_bar["close"] < previous_bar["open"] \
                and current_bar["close"] < current_bar["open"]:
            
            if not in_trade:
                # Open a sell trade
                trade = {"entry_time": current_bar.name, "entry_price": current_bar["open"], "trade_type": "SELL"}
                trades.append(trade)
                in_trade = True

        # Check for trade closing condition
        if in_trade:
            if current_bar["close"] < current_bar["open"]:
                # Close the trade
                trades[-1]["exit_time"] = current_bar.name
                trades[-1]["exit_price"] = current_bar["close"]
                trades[-1]["pnl"] = trades[-1]["entry_price"] - trades[-1]["exit_price"]
                trades[-1]["pnl"] *= lot_size
                in_trade = False

    return trades

def plot_trades(df, trades):
    fig = go.Figure(data=[go.Candlestick(x=df.index,
                                        open=df['open'],
                                        high=df['high'],
                                        low=df['low'],
                                        close=df['close'],
                                        increasing_line_color='orange',
                                        decreasing_line_color='green',
                                        line_width=1)])

    entry_exit_traces = []  # Store entry and exit traces separately

    for trade in trades:
        entry_time = trade["entry_time"]
        entry_price = trade["entry_price"]
        exit_time = trade["exit_time"]
        exit_price = trade["exit_price"]

        entry_trace = go.Scatter(x=[entry_time], y=[entry_price],
                                 mode='markers',
                                 marker=dict(size=10, symbol='triangle-up', color='orange'))
        
        exit_trace = go.Scatter(x=[exit_time], y=[exit_price],
                                mode='markers',
                                marker=dict(size=10, symbol='triangle-down', color='green'))

        entry_exit_traces.append(entry_trace)
        entry_exit_traces.append(exit_trace)

    # Calculate RSI levels
    rsi_levels = df['rsi']
    rsi_oversold = [30] * len(df)
    rsi_overbought = [70] * len(df)

    # Create a subplot with shared x-axis
    subplot_fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05)

    # Add candlestick chart to the first subplot
    subplot_fig.add_trace(go.Candlestick(x=df.index,
                                          open=df['open'],
                                          high=df['high'],
                                          low=df['low'],
                                          close=df['close'],
                                          increasing_line_color='orange',
                                          decreasing_line_color='green',
                                          line_width=1), row=1, col=1)

    # Add entry and exit traces to the first subplot
    for trace in entry_exit_traces:
        subplot_fig.add_trace(trace, row=1, col=1)

    # Add RSI plot to the second subplot
    subplot_fig.add_trace(go.Scatter(x=df.index, y=rsi_levels, name='RSI', line=dict(color='blue')), row=2, col=1)
    subplot_fig.add_trace(go.Scatter(x=df.index, y=rsi_oversold, name='Oversold', line=dict(color='red', dash='dash')), row=2, col=1)
    subplot_fig.add_trace(go.Scatter(x=df.index, y=rsi_overbought, name='Overbought', line=dict(color='green', dash='dash')), row=2, col=1)

    # Update layout of the subplots
    subplot_fig.update_layout(
        title="Backtest Trades with RSI",
        xaxis_title="Time",
        height=800,  # Set the height of the entire plot
        xaxis_rangeslider_visible=False,
        plot_bgcolor='black',
        paper_bgcolor='black',
        font=dict(color='white'),
        showlegend=True,
        autosize=True,
        margin=dict(l=50, r=50, t=50, b=50),
        xaxis_showgrid=False,
        yaxis_showgrid=False,
    )

    subplot_fig.show()


if __name__ == "__main__":
    symbol = "Boom 1000 Index"
    timeframe = mt5.TIMEFRAME_M1
    lot_size = 1.0
    period = 7.0

    # Specify the date range for historical data (adjust as needed)
    start_date = datetime(2023, 8, 16)
    end_date = datetime.now()
    #end_date = datetime(2023, 8, 16)
    # Step 1: Load historical data within the specified date range
    df = get_historical_data(symbol, timeframe, start_date, end_date)

    # Step 2: Calculate RSI
    calculate_rsi(df, period)

    # Step 3: Backtest the strategy
    trades = backtest_strategy(df, lot_size)

    # Step 4: Print the backtest results
    total_pnl = 0
    for trade in trades:
        print(f"Trade Type: {trade['trade_type']}")
        print(f"Entry Time: {trade['entry_time']}, Entry Price: {trade['entry_price']}")
        print(f"Exit Time: {trade['exit_time']}, Exit Price: {trade['exit_price']}")
        print(f"Profit/Loss: {trade['pnl']:.2f}\n")
        total_pnl += trade['pnl']

    print(f"Total Profit/Loss: {total_pnl:.2f}")
    print(f"Total Trades: {len(trades)}")

    # Step 5: Plot the trades
    plot_trades(df, trades)


Retrieved 1279 data points
Trade Type: SELL
Entry Time: 2023-08-15 23:15:00, Entry Price: 10461.738
Exit Time: 2023-08-15 23:15:00, Exit Price: 10461.178
Profit/Loss: 0.56

Trade Type: SELL
Entry Time: 2023-08-15 23:19:00, Entry Price: 10469.304
Exit Time: 2023-08-15 23:19:00, Exit Price: 10468.579
Profit/Loss: 0.73

Trade Type: SELL
Entry Time: 2023-08-15 23:49:00, Entry Price: 10492.78
Exit Time: 2023-08-15 23:49:00, Exit Price: 10492.219
Profit/Loss: 0.56

Trade Type: SELL
Entry Time: 2023-08-16 00:11:00, Entry Price: 10493.655
Exit Time: 2023-08-16 00:11:00, Exit Price: 10493.127
Profit/Loss: 0.53

Trade Type: SELL
Entry Time: 2023-08-16 00:32:00, Entry Price: 10498.588
Exit Time: 2023-08-16 00:32:00, Exit Price: 10497.976
Profit/Loss: 0.61

Trade Type: SELL
Entry Time: 2023-08-16 00:48:00, Entry Price: 10502.536
Exit Time: 2023-08-16 00:48:00, Exit Price: 10501.877
Profit/Loss: 0.66

Trade Type: SELL
Entry Time: 2023-08-16 01:53:00, Entry Price: 10496.541
Exit Time: 2023-08-16 01:

# **BackTest + Trade Visualizer + RSI (Maverick Crash 1000/500/300)**

In [44]:
def backtest_strategy(df, lot_size=0.2):
    trades = []
    in_trade = False

    for i in range(3, len(df)):
        current_bar = df.iloc[i]
        previous_bar = df.iloc[i - 1]
        confirmation_bar = df.iloc[i - 2]

        # Check for buy conditions
        if current_bar["rsi"] < 30 and confirmation_bar["close"] < confirmation_bar["open"] \
                and previous_bar["close"] > previous_bar["open"] \
                and current_bar["close"] > current_bar["open"]:
            
            if not in_trade:
                # Open a buy trade
                trade = {"entry_time": current_bar.name, "entry_price": current_bar["open"], "trade_type": "BUY"}
                trades.append(trade)
                in_trade = True

        # Check for trade closing condition
        if in_trade:
            if current_bar["close"] > current_bar["open"]:
                # Close the trade
                trades[-1]["exit_time"] = current_bar.name
                trades[-1]["exit_price"] = current_bar["close"]
                trades[-1]["pnl"] = trades[-1]["exit_price"] - trades[-1]["entry_price"]
                trades[-1]["pnl"] *= lot_size
                in_trade = False

    return trades

from plotly.subplots import make_subplots

def plot_trades(df, trades):
    fig = go.Figure(data=[go.Candlestick(x=df.index,
                                        open=df['open'],
                                        high=df['high'],
                                        low=df['low'],
                                        close=df['close'],
                                        increasing_line_color='orange',
                                        decreasing_line_color='green',
                                        line_width=1)])

    entry_exit_traces = []  # Store entry and exit traces separately

    for trade in trades:
        entry_time = trade["entry_time"]
        entry_price = trade["entry_price"]
        exit_time = trade["exit_time"]
        exit_price = trade["exit_price"]

        entry_trace = go.Scatter(x=[entry_time], y=[entry_price],
                                 mode='markers',
                                 marker=dict(size=10, symbol='triangle-up', color='orange'))
        
        exit_trace = go.Scatter(x=[exit_time], y=[exit_price],
                                mode='markers',
                                marker=dict(size=10, symbol='triangle-down', color='green'))

        entry_exit_traces.append(entry_trace)
        entry_exit_traces.append(exit_trace)

    # Calculate RSI levels
    rsi_levels = df['rsi']
    rsi_oversold = [30] * len(df)
    rsi_overbought = [70] * len(df)

    # Create a subplot with shared x-axis
    subplot_fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05)

    # Add candlestick chart to the first subplot
    subplot_fig.add_trace(go.Candlestick(x=df.index,
                                          open=df['open'],
                                          high=df['high'],
                                          low=df['low'],
                                          close=df['close'],
                                          increasing_line_color='orange',
                                          decreasing_line_color='green',
                                          line_width=1), row=1, col=1)

    # Add entry and exit traces to the first subplot
    for trace in entry_exit_traces:
        subplot_fig.add_trace(trace, row=1, col=1)

    # Add RSI plot to the second subplot
    subplot_fig.add_trace(go.Scatter(x=df.index, y=rsi_levels, name='RSI', line=dict(color='blue')), row=2, col=1)
    subplot_fig.add_trace(go.Scatter(x=df.index, y=rsi_oversold, name='Oversold', line=dict(color='red', dash='dash')), row=2, col=1)
    subplot_fig.add_trace(go.Scatter(x=df.index, y=rsi_overbought, name='Overbought', line=dict(color='green', dash='dash')), row=2, col=1)

    # Update layout of the subplots
    subplot_fig.update_layout(
        title="Backtest Trades with RSI",
        xaxis_title="Time",
        height=800,  # Set the height of the entire plot
        xaxis_rangeslider_visible=False,
        plot_bgcolor='black',
        paper_bgcolor='black',
        font=dict(color='white'),
        showlegend=True,
        autosize=True,
        margin=dict(l=50, r=50, t=50, b=50),
        xaxis_showgrid=False,
        yaxis_showgrid=False,
    )

    subplot_fig.show()

def plot_histogram(trades):
    pnl_values = [trade.get('pnl', 0) for trade in trades]

    # Filter out zero values (trades that didn't close)
    pnl_values = [pnl for pnl in pnl_values if pnl != 0]

    winning_trades = [pnl for pnl in pnl_values if pnl > 0]
    losing_trades = [pnl for pnl in pnl_values if pnl < 0]

    fig = go.Figure()

    fig.add_trace(go.Histogram(x=winning_trades, opacity=0.75, name="Winning Trades"))
    fig.add_trace(go.Histogram(x=losing_trades, opacity=0.75, name="Losing Trades"))

    fig.update_layout(
        title="Number of Winning and Losing Trades",
        xaxis_title="Profit/Loss",
        yaxis_title="Number of Trades",
        plot_bgcolor='black',
        paper_bgcolor='black',
        font=dict(color='white'),
        showlegend=True,
        autosize=True,
        margin=dict(l=50, r=50, t=50, b=50),
        xaxis_showgrid=True,
        yaxis_showgrid=True,
    )

    fig.update_layout(barmode='overlay')

    fig.show()

def plot_total(trades):
    pnl_values = [trade.get('pnl', 0) for trade in trades]

    # Filter out zero values (trades that didn't close)
    pnl_values = [pnl for pnl in pnl_values if pnl != 0]

    winning_trades = [pnl for pnl in pnl_values if pnl > 0]
    losing_trades = [pnl for pnl in pnl_values if pnl < 0]

    total_winning_trades = len(winning_trades)
    total_losing_trades = len(losing_trades)

    fig = go.Figure()

    fig.add_trace(go.Bar(x=["Winning Trades"], y=[total_winning_trades], name="Winning Trades", marker_color='green'))
    fig.add_trace(go.Bar(x=["Losing Trades"], y=[total_losing_trades], name="Losing Trades", marker_color='red'))

    fig.update_layout(
        title="Number of Winning and Losing Trades",
        xaxis_title="Trade Type",
        yaxis_title="Number of Trades",
        plot_bgcolor='black',
        paper_bgcolor='black',
        font=dict(color='white'),
        showlegend=True,
        autosize=True,
        margin=dict(l=50, r=50, t=50, b=50),
        xaxis_showgrid=False,
        yaxis_showgrid=True,
    )

    fig.show()

if __name__ == "__main__":
    symbol = "Crash 1000 Index"
    timeframe = mt5.TIMEFRAME_M1
    lot_size = 1.0
    period = 7.0

    # Specify the date range for historical data (adjust as needed)
    start_date = datetime(2023, 7, 16)
    end_date = datetime.now()
    #end_date = datetime(2023, 8, 16)
    # Step 1: Load historical data within the specified date range
    df = get_historical_data(symbol, timeframe, start_date, end_date)

    # Step 2: Calculate RSI
    calculate_rsi(df, period)

    # Step 3: Backtest the strategy
    trades = backtest_strategy(df, lot_size)

    # Step 4: Print the backtest results
    total_pnl = 0
    for trade in trades:
        print(f"Trade Type: {trade['trade_type']}")
        print(f"Entry Time: {trade['entry_time']}, Entry Price: {trade['entry_price']}")
        print(f"Exit Time: {trade['exit_time']}, Exit Price: {trade['exit_price']}")
        print(f"Profit/Loss: {trade['pnl']:.2f}\n")
        total_pnl += trade['pnl']

    print(f"Total Profit/Loss: {total_pnl:.2f}")
    print(f"Total Trades: {len(trades)}")

    # Step 5: Plot the trades
    plot_histogram(trades)
    plot_total(trades)
    #plot_trades(df, trades)


Retrieved 45878 data points
Trade Type: BUY
Entry Time: 2023-07-15 23:06:00, Entry Price: 7505.162
Exit Time: 2023-07-15 23:06:00, Exit Price: 7505.579
Profit/Loss: 0.42

Trade Type: BUY
Entry Time: 2023-07-15 23:27:00, Entry Price: 7504.018
Exit Time: 2023-07-15 23:27:00, Exit Price: 7504.444
Profit/Loss: 0.43

Trade Type: BUY
Entry Time: 2023-07-15 23:32:00, Entry Price: 7487.932
Exit Time: 2023-07-15 23:32:00, Exit Price: 7488.413
Profit/Loss: 0.48

Trade Type: BUY
Entry Time: 2023-07-16 00:03:00, Entry Price: 7490.442
Exit Time: 2023-07-16 00:03:00, Exit Price: 7490.904
Profit/Loss: 0.46

Trade Type: BUY
Entry Time: 2023-07-16 00:58:00, Entry Price: 7505.47
Exit Time: 2023-07-16 00:58:00, Exit Price: 7505.897
Profit/Loss: 0.43

Trade Type: BUY
Entry Time: 2023-07-16 02:17:00, Entry Price: 7519.425
Exit Time: 2023-07-16 02:17:00, Exit Price: 7519.87
Profit/Loss: 0.44

Trade Type: BUY
Entry Time: 2023-07-16 03:19:00, Entry Price: 7537.31
Exit Time: 2023-07-16 03:19:00, Exit Price: 75

## **BackTest (RSI under 30 strategy BOOM) + Trade Visualizer (Profitable but high amount lost per trades ❌)**

## To be tested too

In [26]:
# ...

def backtest_new_strategy(df, lot_size=0.2):
    trades = []
    in_trade = False

    for i in range(2, len(df)):
        current_bar = df.iloc[i]
        previous_bar = df.iloc[i - 1]

        # Check for new sell conditions
        if previous_bar["rsi"] > 30 and current_bar["rsi"] <= 30:
            if not in_trade:
                # Open a sell trade
                trade = {"entry_time": current_bar.name, "entry_price": current_bar["open"], "trade_type": "SELL"}
                trades.append(trade)
                in_trade = True

        # Check for trade closing condition
        if in_trade:
            # Calculate the closing time for the trade (1 minute after the trade was executed)
            closing_time = trade["entry_time"] + timedelta(minutes=1)

            if current_bar.name >= closing_time:
                # Close the trade
                trade["exit_time"] = current_bar.name
                trade["exit_price"] = current_bar["close"]
                trade["pnl"] = trade["entry_price"] - trade["exit_price"]
                trade["pnl"] *= lot_size
                in_trade = False

    return trades

def plot_trades(df, trades):
    fig = go.Figure(data=[go.Candlestick(x=df.index,
                                        open=df['open'],
                                        high=df['high'],
                                        low=df['low'],
                                        close=df['close'],
                                        increasing_line_color='orange',
                                        decreasing_line_color='green',
                                        line_width=1)])

    for trade in trades:
        entry_time = trade["entry_time"]
        entry_price = trade["entry_price"]
        exit_time = trade.get("exit_time", None)
        exit_price = trade.get("exit_price", None)

        fig.add_trace(go.Scatter(x=[entry_time],
        y=[entry_price],
        mode='markers',
        marker=dict(size=[10],
        symbol=['triangle-up'],
        color=['orange'],
        line=dict(width=2)),
        name="Entry"))

        if exit_time is not None and exit_price is not None:
            fig.add_trace(go.Scatter(x=[exit_time],
            y=[exit_price],
            mode='markers',
            marker=dict(size=[10],
            symbol=['triangle-down'],
            color=['green'],
            line=dict(width=2)),
            name="Exit"))

    fig.update_layout(
        title="Backtest Trades",
        xaxis_title="Time",
        yaxis_title="Price",
        xaxis_rangeslider_visible=False,
        plot_bgcolor='black',
        paper_bgcolor='black',
        font=dict(color='white'),
        showlegend=True,
        autosize=True,
        margin=dict(l=50, r=50, t=50, b=50),
        xaxis_showgrid=False,
        yaxis_showgrid=False,
    )

    fig.update_layout(width=800, height=800)

    fig.show()

def plot_histogram(trades):
    pnl_values = [trade.get('pnl', 0) for trade in trades]

    # Filter out zero values (trades that didn't close)
    pnl_values = [pnl for pnl in pnl_values if pnl != 0]

    winning_trades = [pnl for pnl in pnl_values if pnl > 0]
    losing_trades = [pnl for pnl in pnl_values if pnl < 0]

    fig = go.Figure()

    fig.add_trace(go.Histogram(x=winning_trades, opacity=0.75, name="Winning Trades"))
    fig.add_trace(go.Histogram(x=losing_trades, opacity=0.75, name="Losing Trades"))

    fig.update_layout(
        title="Number of Winning and Losing Trades",
        xaxis_title="Profit/Loss",
        yaxis_title="Number of Trades",
        plot_bgcolor='black',
        paper_bgcolor='black',
        font=dict(color='white'),
        showlegend=True,
        autosize=True,
        margin=dict(l=50, r=50, t=50, b=50),
        xaxis_showgrid=True,
        yaxis_showgrid=True,
    )

    fig.update_layout(barmode='overlay')

    fig.show()

def plot_total(trades):
    pnl_values = [trade.get('pnl', 0) for trade in trades]

    # Filter out zero values (trades that didn't close)
    pnl_values = [pnl for pnl in pnl_values if pnl != 0]

    winning_trades = [pnl for pnl in pnl_values if pnl > 0]
    losing_trades = [pnl for pnl in pnl_values if pnl < 0]

    total_winning_trades = len(winning_trades)
    total_losing_trades = len(losing_trades)

    fig = go.Figure()

    fig.add_trace(go.Bar(x=["Winning Trades"], y=[total_winning_trades], name="Winning Trades", marker_color='green'))
    fig.add_trace(go.Bar(x=["Losing Trades"], y=[total_losing_trades], name="Losing Trades", marker_color='red'))

    fig.update_layout(
        title="Number of Winning and Losing Trades",
        xaxis_title="Trade Type",
        yaxis_title="Number of Trades",
        plot_bgcolor='black',
        paper_bgcolor='black',
        font=dict(color='white'),
        showlegend=True,
        autosize=True,
        margin=dict(l=50, r=50, t=50, b=50),
        xaxis_showgrid=False,
        yaxis_showgrid=True,
    )

    fig.show()


if __name__ == "__main__":
    symbol = "Boom 1000 Index"
    timeframe = mt5.TIMEFRAME_M1
    lot_size = 1.0
    period = 7

    # Specify the date range for historical data (adjust as needed)
    start_date = datetime(2023, 8, 16)
    end_date = datetime.now()

    # Step 1: Load historical data within the specified date range
    df = get_historical_data(symbol, timeframe, start_date, end_date)

    # Step 2: Calculate RSI
    calculate_rsi(df, period)

    # Step 3: Backtest the new strategy
    new_trades = backtest_new_strategy(df, lot_size)

    # Step 4: Print the backtest results
    total_pnl = 0
    for trade in new_trades:
        print(f"Trade Type: {trade['trade_type']}")
        print(f"Entry Time: {trade['entry_time']}, Entry Price: {trade['entry_price']}")
        print(f"Exit Time: {trade.get('exit_time', 'Not closed')}, Exit Price: {trade.get('exit_price', 'N/A')}")
        print(f"Profit/Loss: {trade.get('pnl', 0):.2f}\n")
        total_pnl += trade.get('pnl', 0)

    print(f"Total Profit/Loss: {total_pnl:.2f}")
    print(f"Total Trades: {len(new_trades)}")
    plot_histogram(new_trades)
    plot_total(new_trades)
    #plot_trades(df, trades)

Retrieved 1221 data points
Trade Type: SELL
Entry Time: 2023-08-15 23:31:00, Entry Price: 10461.464
Exit Time: 2023-08-15 23:32:00, Exit Price: 10460.366
Profit/Loss: 1.10

Trade Type: SELL
Entry Time: 2023-08-16 00:08:00, Entry Price: 10481.12
Exit Time: 2023-08-16 00:09:00, Exit Price: 10494.265
Profit/Loss: -13.14

Trade Type: SELL
Entry Time: 2023-08-16 00:23:00, Entry Price: 10485.976
Exit Time: 2023-08-16 00:24:00, Exit Price: 10484.766
Profit/Loss: 1.21

Trade Type: SELL
Entry Time: 2023-08-16 01:00:00, Entry Price: 10494.916
Exit Time: 2023-08-16 01:01:00, Exit Price: 10493.655
Profit/Loss: 1.26

Trade Type: SELL
Entry Time: 2023-08-16 01:44:00, Entry Price: 10471.685
Exit Time: 2023-08-16 01:45:00, Exit Price: 10470.294
Profit/Loss: 1.39

Trade Type: SELL
Entry Time: 2023-08-16 02:19:00, Entry Price: 10487.494
Exit Time: 2023-08-16 02:20:00, Exit Price: 10486.34
Profit/Loss: 1.15

Trade Type: SELL
Entry Time: 2023-08-16 02:34:00, Entry Price: 10481.559
Exit Time: 2023-08-16 02

# ***BackTest (RSI over 70 strategy Crash) + Trade Visualizer (Profitable ✅)***

### This strategy seems to catch every spike (This should be Impossible)

### Promitting but to be tested wildly before deploying, watch out for performance on demo account and see the logic behind it !!! 

### This look like a simulation of if a trade was taken on every spike

In [45]:
# ...

def backtest_new_strategy(df, lot_size=0.2):
    trades = []
    in_trade = False

    for i in range(2, len(df)):
        current_bar = df.iloc[i]
        previous_bar = df.iloc[i - 1]

        # Check for new sell conditions
        if previous_bar["rsi"] > 70 and current_bar["rsi"] <= 70:
            if not in_trade:
                # Open a sell trade
                trade = {"entry_time": current_bar.name, "entry_price": current_bar["open"], "trade_type": "SELL"}
                trades.append(trade)
                in_trade = True

        # Check for trade closing condition
        if in_trade:
            # Calculate the closing time for the trade (1 minute after the trade was executed)
            closing_time = trade["entry_time"] + timedelta(minutes=1)

            if current_bar.name >= closing_time:
                # Close the trade
                trade["exit_time"] = current_bar.name
                trade["exit_price"] = current_bar["close"]
                trade["pnl"] = trade["entry_price"] - trade["exit_price"]
                trade["pnl"] *= lot_size
                in_trade = False

    return trades

def plot_histogram(trades):
    pnl_values = [trade.get('pnl', 0) for trade in trades]

    # Filter out zero values (trades that didn't close)
    pnl_values = [pnl for pnl in pnl_values if pnl != 0]

    winning_trades = [pnl for pnl in pnl_values if pnl > 0]
    losing_trades = [pnl for pnl in pnl_values if pnl < 0]

    fig = go.Figure()

    fig.add_trace(go.Histogram(x=winning_trades, opacity=0.75, name="Winning Trades"))
    fig.add_trace(go.Histogram(x=losing_trades, opacity=0.75, name="Losing Trades"))

    fig.update_layout(
        title="Number of Winning and Losing Trades",
        xaxis_title="Profit/Loss",
        yaxis_title="Number of Trades",
        plot_bgcolor='black',
        paper_bgcolor='black',
        font=dict(color='white'),
        showlegend=True,
        autosize=True,
        margin=dict(l=50, r=50, t=50, b=50),
        xaxis_showgrid=True,
        yaxis_showgrid=True,
    )

    fig.update_layout(barmode='overlay')

    fig.show()
    
if __name__ == "__main__":
    symbol = "Crash 1000 Index"
    timeframe = mt5.TIMEFRAME_M1
    lot_size = 1
    period = 7

    # Specify the date range for historical data (adjust as needed)
    start_date = datetime(2023, 8, 16)
    end_date = datetime.now()

    # Step 1: Load historical data within the specified date range
    df = get_historical_data(symbol, timeframe, start_date, end_date)

    # Step 2: Calculate RSI
    calculate_rsi(df, period)

    # Step 3: Backtest the new strategy
    new_trades = backtest_new_strategy(df, lot_size)

    # Step 4: Print the backtest results
    total_pnl = 0
    for trade in new_trades:
        print(f"Trade Type: {trade['trade_type']}")
        print(f"Entry Time: {trade['entry_time']}, Entry Price: {trade['entry_price']}")
        print(f"Exit Time: {trade.get('exit_time', 'Not closed')}, Exit Price: {trade.get('exit_price', 'N/A')}")
        print(f"Profit/Loss: {trade.get('pnl', 0):.2f}\n")
        total_pnl += trade.get('pnl', 0)

    print(f"Total Profit/Loss: {total_pnl:.2f}")
    print(f"Total Trades: {len(new_trades)}")
    plot_histogram(new_trades)
    plot_total(new_trades)

Retrieved 1274 data points
Trade Type: SELL
Entry Time: 2023-08-15 23:10:00, Entry Price: 6898.254
Exit Time: 2023-08-15 23:11:00, Exit Price: 6890.545
Profit/Loss: 7.71

Trade Type: SELL
Entry Time: 2023-08-15 23:34:00, Entry Price: 6895.101
Exit Time: 2023-08-15 23:35:00, Exit Price: 6892.182
Profit/Loss: 2.92

Trade Type: SELL
Entry Time: 2023-08-16 00:09:00, Entry Price: 6891.978
Exit Time: 2023-08-16 00:10:00, Exit Price: 6879.711
Profit/Loss: 12.27

Trade Type: SELL
Entry Time: 2023-08-16 01:44:00, Entry Price: 6895.858
Exit Time: 2023-08-16 01:45:00, Exit Price: 6893.288
Profit/Loss: 2.57

Trade Type: SELL
Entry Time: 2023-08-16 02:33:00, Entry Price: 6911.394
Exit Time: 2023-08-16 02:34:00, Exit Price: 6894.504
Profit/Loss: 16.89

Trade Type: SELL
Entry Time: 2023-08-16 03:02:00, Entry Price: 6902.261
Exit Time: 2023-08-16 03:03:00, Exit Price: 6892.356
Profit/Loss: 9.91

Trade Type: SELL
Entry Time: 2023-08-16 04:40:00, Entry Price: 6916.14
Exit Time: 2023-08-16 04:41:00, Exit